In [1]:
########################################################################################################
# Versión 1
# Predecir años 2020-2021 completos según modelos ya creados por estación y atributo
# Guarda la prediccion en un archivo
# nombre archivo predicción: Prediccion_2020-2021_ + COD_ESTACION + ATRIBUTO  ej.: Prediccion_2020-2021_46250048_PM10.csv
########################################################################################################

In [2]:
# Parámetros del algoritmo

# Definir el número de días de la ventana de entrada (días previos a considerar para la predicción)
n = 30

# Definir el número de días a predecir por cada ventana de entrada
ventSalida = 1

# seleccionar lista de atributos a generar modelo de predicción
atributos = ['PM10', 'PM2.5', 'O3', 'NO2', 'SO2']

In [3]:
# librerías necesarias

import pandas as pd
import numpy as np
import plotly 
import plotly.express as px
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Configuración general de las figuras que representaremos
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (20, 10)

import warnings
warnings.filterwarnings("ignore") # specify to ignore warning messages

# desactivar modo ansioso de tensorflow
tf.compat.v1.disable_eager_execution()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path = '/content/drive/MyDrive/datoscsv/'
dfCompleto = pd.read_csv(path + 'Valencia_SeleccionDatos_ParaModelar_v1.csv', sep=';', index_col='fecha', parse_dates=['fecha'])

# Recuperar la fecha como campo
dfCompleto.insert(0, 'FECHA', dfCompleto.index.strftime('%Y-%m-%d'))


In [6]:
def procesar_atributo(atributo, cod_esta, nom_esta, df):
  print(f'Procesando atributo: {atributo}')
  print()
  
  # Preparación de datos de días previos.
  
  # eliminar filas sin el atributo
  df = df[df[atributo].notnull()]
  # Se toman los "n" ultimos días de 2019 + 2020 y 2021 completos para predecir
  # Para poder introducir los datos en una red neuronal hay que transformarlos, ya que solo funciona con arrays
  df2 = df[[atributo]]
  train = df2['2019']     # guardar el 2019
  test = train.iloc[-n:]  # guardar los "n" últimos dias del 2019
  test = test.append(df2['2020':'2021'], ignore_index=False) # añadir a los "n" días del 2019 + el 2020 y 2021 completos
  
  # Se recupera el modelo guardado previamente:

  pathModelos = '/content/drive/MyDrive/modelosGuardados/'
  modelo = keras.models.load_model(pathModelos + 'Modelo_' + str(cod_esta) + '_' + atributo + '.h5')

  # Predicciones
  # Hay que preparar los datos: hacemos bloques de "n" días para realizar la predicción.
  # normalizar datos
  sc = MinMaxScaler(feature_range=(0,1))
  test_scaled = sc.fit_transform(test)

  X_test = []
  for i in range(n,len(test_scaled)):
      X_test.append(test_scaled[i-n:i,0])

  X_test = np.array(X_test)
  X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

  # Realizamos la prediccion y aplicamos normalización inversa para que esté en escala real
  prediccion = modelo.predict(X_test)
  prediccion = sc.inverse_transform(prediccion)

  # Visualización del resultado
  resultados = test.tail(len(test)-n)
  resultados['Prediccion'] = prediccion

  plt.plot(train[atributo], color='green')
  plt.plot(resultados[atributo], color='red', label='Valor real de ' + atributo)
  plt.plot(resultados['Prediccion'], color='blue', label='Predicción de ' + atributo)
  plt.xlabel('Tiempo')
  plt.ylabel('Valor de ' + atributo)
  plt.legend()
  plt.title(nom_esta + ': ' + atributo )
  plt.show()

  from sklearn.metrics import mean_absolute_error
  from sklearn.metrics import mean_squared_error
  from math import sqrt

  # ratios de la predicción
  mae = mean_absolute_error(resultados['Prediccion'], resultados[atributo])
  rmse = sqrt(mean_squared_error(resultados['Prediccion'], resultados[atributo]))

  print(f'Modelo: {cod_esta} - {atributo} / Error cuadrático medio: {rmse:6.3f} / Error absoluto medio: {mae:6.3f}')
  print('========================================')
  print()

  plt.plot(resultados[atributo], color='red', label='Valor real de ' + atributo)
  plt.plot(resultados['Prediccion'], color='blue', label='Predicción de ' + atributo)
  plt.xlabel('Tiempo')
  plt.ylabel('Valor de ' + atributo)
  plt.legend()
  plt.title(nom_esta + ': ' + atributo )
  plt.show()
  
  # Recuperar la fecha como campo en predicción
  resultados.insert(0, 'FECHA', resultados.index.strftime('%Y-%m-%d'))
  print(resultados)
  
  # Guardar la prediccion
  pathPrediccion = '/content/drive/MyDrive/predicciones/'
  (pathModelos + 'Prediccion_' + str(cod_esta) + '_' + atributo + '.csv')
  resultados.to_csv(pathPrediccion + 'Prediccion_2020-2021_' + str(cod_esta) + '_' + atributo + '.csv', header=True, sep=';', index=False)  
    

In [7]:
def procesar_estacion(cod_esta, nom_esta, df):
  print(f'Procesando estacion: {cod_esta} - {nom_esta}')
  print()
  # seleccionar los datos de la estacion a procesar
  df = df[df['COD_ESTACION']==cod_esta]
  # print(df)
  # procesar los atributos de la estación
  for atrib in atributos:
    procesar_atributo(atrib, cod_esta, nom_esta, df)

In [8]:
# Proceso principal
import time
start = time.time()

cod_estaciones = dfCompleto['COD_ESTACION'].unique()
nom_estaciones = dfCompleto['NOM_ESTACION'].unique()
for cod_esta, nom_esta in zip(cod_estaciones, nom_estaciones):
  procesar_estacion(cod_esta, nom_esta, dfCompleto)

end = time.time()
print('Segundos transcurridos: ', end - start)

Output hidden; open in https://colab.research.google.com to view.